In [1]:
# imports
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import datetime # one module for working with dates and times
import json #this solution saves and opens json files. You may use a different file type and change the import accordingly

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("600x400") # set the default window size
        self.title('Notebook') #set the default window title
        self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        self.notes = [] # redundant - delete
        self.note = {}

        # create a frame called frame_main that covers the entire window.
        # not required for full credit
        # all other objects will be placed in frame_main instead of the window itself
        self.frame_main = tk.Frame(self)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')
        
        # create a frame in the new window that covers the entire window
        self.frame_notes = tk.Frame(self.frame_main)
        self.frame_notes.grid(row=1, column=3, rowspan=6, sticky='w')
        self.frame_notes.config(bg='gray') 
        
        #define some buttons and put them in a grid in frame_main
        # create new note button - opens a new note form
        btn01=tk.Button(self.frame_main, text='Create New Note', command=self.new_note)
        btn01.grid(padx=10, pady=10, row=1, column=1)
        
        # open notebook button - opens a notebook and displays its notes in the window
        btn02=tk.Button(self.frame_main, text='Open Notebook', command=self.open_notebook)
        btn02.grid(padx=10, pady=10, row=2, column=1) 
        
        # save notebook button - saves the notebook and refreshes the notes display
        btn03=tk.Button(self.frame_main, text='Save Notebook\nand Refresh', command=self.save_notebook)
        btn03.grid(padx=10, pady=10, row=3, column=1) 
        
        # quit button
        btn04=tk.Button(self.frame_main, text='Quit', command=self.destroy)
        btn04.grid(padx=10, pady=10, row=4, column=1)

    # define methods corresponding to the button commands
    
    # new_note opens a new toplevel window with a note form.
    # when the new note's submit button is pressed, the new note is added to the self.notebook
    # attribute in the main_window object.
     
       
    def new_note(self): # opens a new note form
        note_form = NoteForm(self, self.notebook, self.notes)
        return None

    def clear_frame(self, target_frame): # method for clearing old content from the frame
        for widgets in target_frame.winfo_children():
            widgets.destroy()

    def show_notes(self): # generates note objects and displays them in the main window
        self.clear_frame(self.frame_notes) # clears any previous display
        self.notes = [] # resets the notes object list
        for note in self.notebook: # create new note objects from the notebook and display them
            new_note = MakeNote(master=self.frame_notes, note_dict=note)
            self.notes.append(new_note)
            new_note.pack(padx=10, pady=10)
            new_note.config(height= 3, width=40, wraplength=200, justify=tk.LEFT)
        return None
    
    def open_notebook(self):
        # this opens json files. You may use different file types.
        filepath = filedialog.askopenfilename(initialdir="C:\\Users\\duran\\INST326\\TestSolution",
                                         filetypes=[("json files", "*.json"), 
                                                    ("csv files", ".csv"),
                                         ("all files", "*.*")])
        file = open(filepath, "r")
        self.notebook = json.load(file) # load the json file into self.notebook as a list of dictionaries
        file.close()
       
        self.show_notes() # once the file is loaded, call the method to display the notes in the window
        return None
    
    def save_notebook(self):
        # the following code saves the notebook as a json file. You  may use different file types
        file = filedialog.asksaveasfile(initialdir="C:\\Users\\duran\\INST326\\TestSolution",
                                              defaultextension=".json", 
                                        title="notebook01",
                                              filetypes=[("json file", ".json"),
                                             ("all files", ".*")])
        
        json_out = json.dumps(self.notebook, indent=2)
        file.write(json_out)
        
        self.show_notes() # this refreshes the notes display in the main window
        return None


# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, notebook, notes): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window
        # set the new window's default parameters
        self.geometry("600x400") 
        self.title('New Note')
        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')

        
        #define default dummy text (for development purposes only)
        default_note = {"title":"new note title",
                     "text":"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Etiam sit amet suscipit mi, non porttitor mauris. Aliquam in lorem risus. Proin mauris mauris, varius ac vulputate sed, tempor nec lacus. Morbi sodales turpis in placerat semper. Donec bibendum blandit ante sit amet hendrerit.", 
                    "link":"If there is a link with this note enter it here.",
                    "tags":"enter hashtags here",
                    "meta":"metadata added at submission"}
                    
        note = default_note # provided in anticipation of note editing functionality
        
        #define self.notebook as the notebook passed from the main window
        self.notebook = notebook
        #self.notes = notes (not necessary)
        
        # create some labels and put them in the grid
        # we are using the grid layout. Notice the sticky='e' attribute. 
        # this causes the label to 'stick' to the 'east' side of the grid cell
        title_label = tk.Label(self.frame_main, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')

        text_label = tk.Label(self.frame_main, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')

        link_label = tk.Label(self.frame_main, bg='light gray', text='Note Link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')

        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')

        # create our note title entry field
        self.note_title = tk.Entry(self.frame_main, width=80)
        self.note_title.grid(padx=10, pady=10, row=1, column=1, sticky='w')
        self.note_title.insert(0, note["title"]) # adds default text (useful during development)

        # create our note text field
        self.note_text = tk.Text(self.frame_main, height=10, width=60)
        self.note_text.grid(padx=10, pady=10, row=2, column=1)
        self.note_text.insert('1.0', note["text"]) # adds default text (useful during development)

        # create our note link entry field
        self.note_link = tk.Entry(self.frame_main, width=80)
        self.note_link.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        self.note_link.insert(0, note["link"]) # adds default text (useful during development)

        # create our note link entry field
        self.note_tags = tk.Entry(self.frame_main, width=80)
        self.note_tags.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        self.note_tags.insert(0, note["tags"]) # adds default text (useful during development)

        # create our note meta field if you want to add edit functionality
#         self.note_meta = tk.Entry(self.frame_main, width=80)
#         self.note_meta.grid(padx=10, pady=10, row=5, column=1, sticky='w')
#         self.note_meta.insert(0, note["meta"]) # adds default text (useful during development)
        

        # note that the parameters for the Entry box and Text box are slightly different.
        # The user can create multiple notes with the same note form. Each time the 'submit'
        # button is pressed, a new note is added to the notebook.

        b1 = tk.Button(self.frame_main, text='submit', command=self.submit)
        b1.grid(padx=10, pady=10, row=6, column=1, sticky='w')

        b5 = tk.Button(self.frame_main, text='close', command=self.destroy)
        b5.grid(padx=10, pady=10, row=6, column=0) 
       

    def submit(self):
        # calculate the date and time information for the meta field
        now = datetime.datetime.now() # gets the current date and time
        local_now = now.astimezone() # shows the local time and the GMT offset
        local_tz = local_now.tzinfo 
        created = datetime.datetime.now()
        
        # get all the input values and put them into a dictionary along with the metadata
        title = self.note_title.get()
        text = self.note_text.get('1.0', 'end').strip('\n')
        link = self.note_link.get()
        tags = self.note_tags.get()
        meta = f'created {created}, {local_tz}'
        note_dict = {'title':title, 'text':text, 'link':link, 'tags':tags, 'meta':meta}
        
        # add the dictionary to the notebook
        self.notebook.append(note_dict)
        return None

# The MakeNote class creates a new note object.
# The MakeNote class is a subclass of the tk.Button class, 
# which means each note instance has tk.Button functionality

class MakeNote(tk.Button):
    def __init__(self, master=None, note_dict=None): # the arguments on this line
        # are inbound, meaning we pass them when we instantiate the object
        super().__init__(master) # on this line we call the __init__ method of tk.Button and pass
        # the master attribute to it. This gives us all the button attributes and functionality
        
        # define note attributes
        self.title = note_dict['title']
        self.text = note_dict['text']
        self.link = note_dict['link']
        self.tags = note_dict['tags']
        self.meta = note_dict['meta']

        # configure note button; this creates a button with two lines of text
        self.config(bg='light gray', text=f"{self.title}\n{self.meta}")
        
        # Bind mouse events
        self.bind("<Enter>", self.on_hover)
        self.bind("<Leave>", self.on_leave)
        self.bind("<Button-1>", self.note_open)

    def on_hover(self, event): # change the background when the cursor hovers over it
        self.config(bg="lightblue")  

    def on_leave(self, event): # change back when not hovering
        self.config(bg="light gray")  # Restore original color
        
    def note_open(self, event): # on mouse click, open note in new top window
        
        # create a new top window
        self.note_window = tk.Toplevel(main_window, bg="light gray", height=600, width=600)
        self.note_window.title(self.title)
        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self.note_window)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')
        
        # create labels in the frame
        title_label = tk.Label(self.frame_main, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')
        title_content = tk.Label(self.frame_main, bg='light gray', text=self.title, wraplength=400, justify=tk.LEFT)
        title_content.grid(padx=10, pady=10, row=1, column=1, sticky='w')        

        text_label = tk.Label(self.frame_main, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')
        text_content = tk.Label(self.frame_main, bg='light gray', text=self.text, wraplength=400, justify=tk.LEFT)
        text_content.grid(padx=10, pady=10, row=2, column=1, sticky='w')

        link_label = tk.Label(self.frame_main, bg='light gray', text='Note Link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')
        link_content = tk.Label(self.frame_main, bg='light gray', text=self.link, wraplength=400, justify=tk.LEFT)
        link_content.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')
        tag_content = tk.Label(self.frame_main, bg='light gray', text=self.tags, wraplength=400, justify=tk.LEFT)
        tag_content.grid(padx=10, pady=10, row=4, column=1, sticky='w')

        meta_label = tk.Label(self.frame_main, bg='light gray', text='Note Meta:')
        meta_label.grid(padx=10, pady=10, row=5, column=0, sticky='e')
        meta_content = tk.Label(self.frame_main, bg='light gray', text=self.meta, wraplength=400, justify=tk.LEFT)
        meta_content.grid(padx=10, pady=10, row=5, column=1, sticky='w')        

        # create a button to close the note window
        b10 = tk.Button(self.frame_main, text='close', command=self.note_window.destroy)
        b10.grid(padx=10, pady=10, row=6, column=0)
        

# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()



Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\duran\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1962, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\duran\AppData\Local\Temp\ipykernel_9340\3153260724.py", line 83, in open_notebook
    file = open(filepath, "r")
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\duran\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py", line 310, in _modified_open
    return io_open(file, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: ''
